In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!pip install mediapipe
!pip install tensorflow
!pip install tqdm
# !pip install nvidia-ml-py3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense
from tqdm import tqdm
# from sklearn.model_selection import train_test_split
import pickle

2024-04-21 15:11:41.199928: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 15:11:41.264173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 15:11:41.264209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 15:11:41.265514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-21 15:11:41.274035: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 15:11:41.274651: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def extract_keypoints(results):
    """
    Processes and organizes the keypoints detected from the pose estimation model 
    to be used as inputs for the exercise decoder models
    
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # for pos in pose:
    #     print("Pos: ", pos)
    return pose

In [3]:


# Function to extract landmarks from a video file
def extract_landmarks(video_path):
    # Use Mediapipe to extract landmarks from each frame of the video
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    cap = cv2.VideoCapture(video_path)
    landmarks = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        landmark_list = extract_keypoints(results)
        landmarks.append(landmark_list)
        print("Landmark List: ", landmark_list)
    cap.release()
    return landmarks

# Path to the directory containing the gym exercise videos
data_dir = "../dataset"

result_dir = "./WVE"

# List of gym exercise classes (subfolders in the data directory)
exercise_classes = os.listdir(data_dir)

print("Preparing Dataset")
# Prepare the dataset
X, y = [], []
for i, exercise_class in enumerate(exercise_classes):
    class_dir = os.path.join(data_dir, exercise_class)

    output_dir = os.path.join(result_dir, exercise_class)
    result_file_path_x = output_dir + "+X.txt"
    # result_file_path_y = output_dir + "+y.txt"

    filenamex = exercise_class + "+X.txt"

    if filenamex in os.listdir(result_dir):
      continue
    print(class_dir)

    xin, yin = [], []
    for video_file in tqdm(os.listdir(class_dir), desc="Iterating..."):
        video_path = os.path.join(class_dir, video_file)
        landmarks = extract_landmarks(video_path)
        xin.append(landmarks)
        # yin.append(i)
        X.append(landmarks)
        # y.append(i)  # Use the class index as the label
        
    with open(result_file_path_x, "wb") as filex:
      pickle.dump(xin, filex)
    # with open(result_file_path_y, "wb") as filey:
    #   pickle.dump(yin, filey)


Preparing Dataset


FileNotFoundError: [Errno 2] No such file or directory: './WVE'

In [6]:
# X, y = [], []
# for exercise_class in exercise_classes:
#     print(exercise_class)
#     output_dir = os.path.join(result_dir, exercise_class)
#     result_file_path_x = output_dir + "+X.txt"
#     result_file_path_y = output_dir + "+y.txt"

#     with open(result_file_path_x, "rb") as filex:
#       for inpx in pickle.load(filex):
#         X.append(inpx)
#     with open(result_file_path_y, "rb") as filey:
#       for inpy in pickle.load(filey):
#         y.append(inpy)


In [7]:
# with open("./drive/MyDrive/Workout_Extract_Complete+X.txt", "wb") as filewx:
#     pickle.dump(X, filewx)
# with open("./drive/MyDrive/Workout_Extract_Complete+y.txt", "wb") as filewy:
#     pickle.dump(y, filewy)

In [8]:
# print(X)

In [9]:
# print("splitting Dataset")
# # Split the dataset into training, validation, and test sets
# X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=42)

# num_landmarks = 33
# print("Padding or truncating dataset")
# # Pad or truncate sequences to ensure they have the same length
# max_seq_length = max([len(seq) for seq in X_train])
# X_train = [seq[:max_seq_length] + [[0] * num_landmarks] * (max_seq_length - len(seq)) for seq in X_train]
# X_val = [seq[:max_seq_length] + [[0] * num_landmarks] * (max_seq_length - len(seq)) for seq in X_val]
# X_test = [seq[:max_seq_length] + [[0] * num_landmarks] * (max_seq_length - len(seq)) for seq in X_test]

# # Convert to numpy arrays
# X_train = np.array(X_train)
# X_val = np.array(X_val)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_val = np.array(y_val)
# y_test = np.array(y_test)

# print("Defining the model")
# # Define the model
# model = Sequential([
#     Conv1D(64, 3, activation='relu', input_shape=(max_seq_length, num_landmarks)),
#     LSTM(64),
#     Dense(len(exercise_classes), activation='softmax')
# ])

# print("compiling the model")
# # Compile the model
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# print("Training the model")
# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# print("evaluating the model")
# # Evaluate the model
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


In [10]:
# exercises_list = ['barbell biceps curl','bench press','chest fly machine','deadlift','decline bench press',
#                   'hammer curl','hip thrust','incline bench press','lat pulldown','lateral raise',
#                   'leg extension','leg raises','plank','pull Up','push-up','romanian deadlift',
#                   'russian twist','shoulder press','squat','t bar row','tricep Pushdown','tricep dips']
